# 2.23 Регулярные выражения Часть 2
[Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html)

Основное преимущество - шаблонный поиск (маска, а не конкретный сивол)
## Базовые выражения для создания структурных объектов

In [25]:
import re,pandas  as pd
import warnings; warnings.simplefilter('ignore')

### Одиночный поиск
- обычные символы (!не метасимволы масок) соответствуют сами себе 
- `.` означает любой одиночный символ, кроме `\n`
- `\w` == `[a-zA-Z0-9_]`
- `\W` != `[a-zA-Z0-9_]`
- `\b` отмечает границу между словом и не словом
- `\s` == `[\n\r\t\f]`
- `\S` != `[\n\r\t\f]`
- `\t,\n,\r` табуляция, новая строка, возврат
- `\d` == `[0-9]`
- `^,$` начало и конец строки
- `\` запрещает специализацию метасимвола
- `?` после буквы означает, что предыдущая буква не обязательна

In [75]:
# игнорирование специализации метасимволов 
strings=['123-4','1-234','12-34','[]',
         '[]', '*','1|2','()']
patterns=[r'[a-]',r'[-a]',r'[a\-b]',r'[[]]',
          r'[\[\]]',r'[(+*]',r'[(+*|]',r'[)(]']
for (pattern,string) in list(zip(patterns,strings)):
    print(f'{(pattern,string)}',re.search(pattern,string))

('[a-]', '123-4') <re.Match object; span=(3, 4), match='-'>
('[-a]', '1-234') <re.Match object; span=(1, 2), match='-'>
('[a\\-b]', '12-34') <re.Match object; span=(2, 3), match='-'>
('[[]]', '[]') <re.Match object; span=(0, 2), match='[]'>
('[\\[\\]]', '[]') <re.Match object; span=(0, 1), match='['>
('[(+*]', '*') <re.Match object; span=(0, 1), match='*'>
('[(+*|]', '1|2') <re.Match object; span=(1, 2), match='|'>
('[)(]', '()') <re.Match object; span=(0, 1), match='('>


In [26]:
# можно использовать без привлечения модуля re
df=pd.DataFrame(
    {'товар':['крем для рук','жидкое мыло','гель для душа','расческа'],
              'цена': ['$11.35','€2.44','$4.25','€2.12']})
# цены в долларах - символ в начале строки
df[df['цена'].str.match('^\$')]

,товар,цена
0,крем для рук,$11.35
2,гель для душа,$4.25


In [27]:
df=pd.DataFrame(
    {'номер телефона': ['555-1234','(555)555-4321','555.555.6789']})
# удалить все символы, не являющиеся цифрами, заменив пустой строкой
df['номер телефона'].str.replace(r'\D+','')

0       5551234
1    5555554321
2    5555556789
Name: номер телефона, dtype: object

In [28]:
df=pd.DataFrame(
    {'ФИО':['Смирнова Мария Ивановна','Колесников Игорь Сергеевич', 
            'Стрельцова Маргарита Викторовна']})
# создание новых полей путем разделения по пробелу
df[['фамилия','имя','отчество']]=df['ФИО'].str.split(r'\s+',expand=True)
df

,ФИО,фамилия,имя,отчество
0,Смирнова Мария Ивановна,Смирнова,Мария,Ивановна
1,Колесников Игорь Сергеевич,Колесников,Игорь,Сергеевич
2,Стрельцова Маргарита Викторовна,Стрельцова,Маргарита,Викторовна


### Фрагментарный поиск (слова или наборы символов)
#### Применение повторов и логических операторов
- элементарные обозначения
    - `+` => 1 или более вхождений шаблона слева от символа
    - `*` => 0 или более вхождений шаблона слева от символа
    - `?` => 0 или 1 вхождение шаблона слева от символа
    - `|` => или
    - `[]` => устанавливает класс символов
    - `{}` => определяет количество повторений
    - `()` => формирует группы
    - `<>` => организует именованные группы
- усложненные варианты
    - `[a-e]` => `[abcde]`
    - `[^abc]` => не должны быть a,b,c
    - `a{n,m}` => a встречается не меньше n и не больше m раз
    - `(x|y)` => поиск х или у
    - `\b[^x\s]\w+` => в начале слова не стоит х
    - `([a-z])\1+` => есть повторяющиеся буквы в слове

In [29]:
# в выражении допустимы комментарии
re.search('def(?#Это комментарий) ...', 'abc def ghi')

<re.Match object; span=(4, 11), match='def ghi'>

In [30]:
# \ - самый многовариантный символ
for string in ['a\b','a\\b']:
    print(string,re.search('\\\\',string))
    print(string,re.search(r'\\\\',string))

a None
a None
a\b <re.Match object; span=(1, 2), match='\\'>
a\b None


In [33]:
# неоднозначная специализация: \A,^ - начало строки, [^] - отрицание
# [#:^] - нет специализации; $,\Z - конец строки
strings=['привет','эй привет','привет как дела','привет ^',
         'привет, привет\n','как дела? привет\n','алло, привет']
patterns=[r'\Aпривет',r'^привет',r'[^привет]..',r'[#:^]',
          r'привет$',r'привет\Z',r'привет\Z']
for (pattern,string) in list(zip(patterns,strings)):
    print(f'{(pattern,string)}',re.search(pattern,string))

('\\Aпривет', 'привет') <re.Match object; span=(0, 6), match='привет'>
('^привет', 'эй привет') None
('[^привет]..', 'привет как дела') <re.Match object; span=(6, 9), match=' ка'>
('[#:^]', 'привет ^') <re.Match object; span=(7, 8), match='^'>
('привет$', 'привет, привет\n') <re.Match object; span=(8, 14), match='привет'>
('привет\\Z', 'как дела? привет\n') None
('привет\\Z', 'алло, привет') <re.Match object; span=(6, 12), match='привет'>


In [31]:
# базовые примеры наборов символов
# первый и последний символ, количество пропущенных
pattern='^a....z$'
strings=['abcdez','az','aaazzz']
for string in strings:
    result=re.search(pattern,string)
    if result:
        print(f"поиск в строке {string} успешен: {result.group()}")

поиск в строке abcdez успешен: abcdez
поиск в строке aaazzz успешен: aaazzz


In [32]:
# базовые примеры наборов символов
string='привееет'
for pattern in [r'еее',r'вет',r'...т']:
    print(re.search(pattern,string))
string='привеее1111т'
for pattern in [r'\d\d\d',r'\w\w\w',r'\w\w\d\d']:
    print(re.search(pattern,string))

<re.Match object; span=(4, 7), match='еее'>
None
<re.Match object; span=(4, 8), match='ееет'>
<re.Match object; span=(7, 10), match='111'>
<re.Match object; span=(0, 3), match='при'>
<re.Match object; span=(5, 9), match='ее11'>


In [34]:
# поиск любого контекста из списка
for string in ["x a b c d","xy","ab cd"]:
    if re.search(r"(x|y)",string):
        print(f"В строке '{string}' есть символы x или y")
    else:
        print(f"В строке '{string}' нет символов x или y")

В строке 'x a b c d' есть символы x или y
В строке 'xy' есть символы x или y
В строке 'ab cd' нет символов x или y


In [35]:
# все слова, не начинающиеся с 'р'
pattern=r'\b[^р\s]\w+'
string='астра роза тюльпан: ромашка111'
re.findall(pattern,string)

['астра', 'тюльпан']

In [36]:
# ограничение по количеству символов
for i in range(1, 6):
    string=f"@{'-'*i}@"
    print(f"{i}  {string:10}  {re.search('@-{2,4}@',string)}")

1  @-@         None
2  @--@        <re.Match object; span=(0, 4), match='@--@'>
3  @---@       <re.Match object; span=(0, 5), match='@---@'>
4  @----@      <re.Match object; span=(0, 6), match='@----@'>
5  @-----@     None


In [37]:
# замена повторов подряд и формирование новой строки
string='aabbbccdddddddffffffbbbbbbbqqq'
print(re.sub(r'([a-z])\1+',r'\1',string))
# сравним со строковым методом
print(''.join(set(string)))

abcdfbq
fabqcd


In [38]:
# удаление повторов слов
def remove_word_duplicates(string):
    pattern=r'\b(\w+)(?:\W+\1\b)+' 
    return re.sub(pattern,r'\1',string,flags=re.IGNORECASE)
for string in ["How are are you?",
               "Kaggle is the the best platform to learn.",
               "Programming is fun fun."]:
    print(remove_word_duplicates(string))


How are you?
Kaggle is the best platform to learn.
Programming is fun.


In [39]:
# поиск слов или фраз без учета регистра 
# (?i) обеспечивает игнорирование
pattern='(?i)(\W|^)(привет|здравствуй|здорово|доброе\sутро)(\W|$)'
string='Привет! Как дела, как поживаешь? Прекрасное утро.'
re.findall(pattern,string)

[('', 'Привет', '!')]

In [40]:
# усложним задачу с таблицей данных
df=pd.DataFrame(
    {'ФИО':['Смирнова, Мария Ивановна','Колесников, Игорь Сергеевич', 
            'Стрельцова, Маргарита Викторовна'],
      'возраст':[30,25,40],
      'тарифная ставка':[50000,60000,70000]})
# разделение и реорганизация полей таблицы
df[['фамилия','имя','отчество']]=\
df['ФИО'].str.extract(r'^(\w+),\s+(\w+)\s+(\w+)$')
df=df[['имя','фамилия','возраст','тарифная ставка']]
df

,имя,фамилия,возраст,тарифная ставка
0,Мария,Смирнова,30,50000
1,Игорь,Колесников,25,60000
2,Маргарита,Стрельцова,40,70000


In [41]:
# с использованием анонимной функции
df=pd.DataFrame({'сообщения':
    ['Электронный адрес для контактов contacts@gmail.com',
     'Прошу переслать сообщение клиенту d_ivanov@sagemath.net',
     'Отправь мне фото сюда: photo@example.org']})
df['email']=df['сообщения'].apply(
    lambda s: re.findall(r'\b[\w.-]+?@\w+?\.\w+?\b',s)[0])
df

,сообщения,email
0,Электронный адрес для контактов contacts@gmail...,contacts@gmail.com
1,Прошу переслать сообщение клиенту d_ivanov@sag...,d_ivanov@sagemath.net
2,Отправь мне фото сюда: photo@example.org,photo@example.org


In [42]:
df=pd.DataFrame({'обозначение':['IL','IN','OH','MI'],
                 'название штата':['Illinois','Indiana','Ohio','Michigan'],
                 'город':['Chicago','Indianapolis','Columbus','Detroit']})

# контекстный поиск 'in', 'chi' в строковых литералах таблицы
df[df.apply(
    lambda row: any(
        [bool(re.search(r'in|chi',str(col))) for col in row]),axis=1)]

,обозначение,название штата,город
0,IL,Illinois,Chicago
3,MI,Michigan,Detroit


### Применение продвинутой разметки структуры

In [43]:
# пропустить ненужную информацию (группу) в поиске
result=re.search(
    '(\w+),(?:\w+),(\w+)','абв,где,ёжз')
result.groups()

('абв', 'ёжз')

#### `(?<set_flags>-<remove_flags>:<regex>)`

- устанавливает или удаляет значение флага на время действия группы

In [51]:
# игнорирование регистра не распространяется на группу (?-i:абв)
re.search('(?-i:абв)где','абвГДЕ',re.IGNORECASE),\
re.search('(?-i:абв)где','АБВГДЕ',re.IGNORECASE)

(<re.Match object; span=(0, 6), match='абвГДЕ'>, None)

#### Регулирование направления поиска
- `(?=<lookahead_regex>)` позитивный поиск вперед
- `(?!<lookahead_regex>)` негативный поиск вперед
- `(?<=<lookbehind_regex>)` позитивный поиск назад
- `(?<!<lookbehind_regex>)` негативный поиск назад


In [86]:
strings=['abcde','abc1','abcd','abc1',
         '123def', 'cdef','123de','abcd']
patterns=[r'abc(?=[a-z][a-z])',r'abc(?=[a-z])',
          r'abc(?![a-z])',r'abc(?![a-z])',
          r'(?<=abc)def',r'(?<=[a-z])def',
          r'(?<!abc)de',r'(?<![a-z])bcd']
for (pattern,string) in list(zip(patterns,strings)):
    print(f'{(pattern,string)}',re.search(pattern,string))

('abc(?=[a-z][a-z])', 'abcde') <re.Match object; span=(0, 3), match='abc'>
('abc(?=[a-z])', 'abc1') None
('abc(?![a-z])', 'abcd') None
('abc(?![a-z])', 'abc1') <re.Match object; span=(0, 3), match='abc'>
('(?<=abc)def', '123def') None
('(?<=[a-z])def', 'cdef') <re.Match object; span=(1, 4), match='def'>
('(?<!abc)de', '123de') <re.Match object; span=(3, 5), match='de'>
('(?<![a-z])bcd', 'abcd') None


In [83]:
# просмотр вперед поиска VS переход вперед поиска
[[re.search('abc(?=[a-z])(?P<ch>.)','abcde'),
  re.search('abc(?=[a-z])(?P<ch>.)','abcde').group('ch')],
 [re.search('abc([a-z])(?P<ch>.)','abcde'),
  re.search('abc([a-z])(?P<ch>.)','abcde').group('ch')]]

[[<re.Match object; span=(0, 4), match='abcd'>, 'd'],
 [<re.Match object; span=(0, 5), match='abcde'>, 'e']]

In [44]:
# присвоение имен группам
result=re.search(
    '(?P<w1>\w+),(?P<w2>\w+),(?P<w3>\w+)','абв,где,ёжз')
result.group('w1','w2','w3'),result.groups()

(('абв', 'где', 'ёжз'), ('абв', 'где', 'ёжз'))

In [45]:
# одинаковый результат поиска 
# повторяющегося через запятую и пробел слова
re.search(r'(\w+), \1', 'привет, привет'),\
re.search(r'(?P<word>\w+), (?P=word)', 'привет, привет')

(<re.Match object; span=(0, 14), match='привет, привет'>,
 <re.Match object; span=(0, 14), match='привет, привет'>)

In [46]:
# создание именованных наборов символов
# широкий спектр форматов дат
pattern=r'(?P<day>\d{1,2}) ?[-/] ?(?P<month>\d{1,2}) ?[-/] ?(?P<year>\d{2,4})'
for string in ['1-12/ 2023','01/07/22','12-01-21','15 /2-2020']:
    print(f"day: {re.search(pattern,string).group('day')}")
    print(f"month: {re.search(pattern,string).group('month')}")
    print(f"year: {re.search(pattern,string).group('year')}")

day: 1
month: 12
year: 2023
day: 01
month: 07
year: 22
day: 12
month: 01
year: 21
day: 15
month: 2
year: 2020


#### Условный поиск
`(?(<n>)<yes-regex>|<no-regex>)` 
- соответствует `<yes-regex>`, если существует группа с номером `<n>`
- в противном случае он соответствует `<no-regex>`

`(?(<name>)<yes-regex>|<no-regex>)`
- соответствует `<yes-regex>`, если существует группа с именем `<name>`
- в противном случае он соответствует `<no-regex>`

In [47]:
pattern=r'^(english - spanish: )?hello - (?(1)hola|привет)'
strings=['english - spanish: hello - hola',
         'english - spanish: hello - привет',
         'hello - hola','hello - привет']
for string in strings:
    print(re.search(pattern,string))

<re.Match object; span=(0, 31), match='english - spanish: hello - hola'>
None
None
<re.Match object; span=(0, 14), match='hello - привет'>


### Примеры "перевода" регулярных выражений
[regex101.com => python](https://regex101.com/library?search=&orderBy=MOST_RECENT&filterFlavors=python)

#### `r"(?<=\>).+(?=\<)"`
- Positive Lookbehind `(?<=\>)` 
- Assert that the Regex below matches
    - `\>` matches the character > with index 62<sub>10</sub> (3E<sub>16</sub>  or 76<sub>8</sub> ) literally (case sensitive)
    - `.` matches any character (except for line terminators)
    - `+` matches the previous token between one and unlimited times, as many times as possible, giving back as needed (greedy)
- Positive Lookahead `(?=\<)`
- Assert that the Regex below matches
    - `\<` matches the character < with index 60<sub>10</sub>  (3C<sub>16</sub>  or 74<sub>8</sub> ) literally (case sensitive)

In [48]:
# текст между символами html тегов
string="<text class='test'> test messages </text>"
pattern=r"(?<=\>).+(?=\<)"
re.search(pattern,string).group()

' test messages '

[Regular Expressions: Regexes in Python](https://realpython.com/regex-python/)

In [89]:
# re.VERBOSE позволяет прокомментировать каждое действие
pattern=r'''
^               # Start of string
(\(\d{3}\))?    # Optional area code
\s*             # Optional whitespace
\d{3}           # Three-digit prefix
[-.]            # Separator character
\d{4}           # Four-digit line number
$               # Anchor at end of string
'''
re.search(pattern,'414.9229',re.VERBOSE)

<re.Match object; span=(0, 8), match='414.9229'>